# 03 - Unsupervised Learning (K-Means Clustering)

This notebook covers:
- Feature selection for clustering
- K-means clustering implementation
- Optimal k selection (elbow method, silhouette analysis)
- Cluster profiling and interpretation
- Student segmentation analysis